In [2]:
import numpy as np
import pandas as pd
import torch
import re
from transformers import AutoModel, AutoTokenizer, RobertaForSequenceClassification


In [78]:
df = pd.read_csv("D:/Study/Data Analyst/LongTH/Level 2/Class 9/Comments.csv",dtype = {'col11': str})

df = df[['_id','user_id','comment_on','object_id','content','device']]
df.head(5)


M:\TEMP\ipykernel_13216\1698841316.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:/Study/Data Analyst/LongTH/Level 2/Class 9/Comments.csv",dtype = {'col11': str})


,_id,user_id,comment_on,object_id,content,device
0,555466761a77ce5d65ea062a,12,event,552b2f0817dc135c89d7d6a2,"Chương trình \ Tuyệt cú mèo\ luôn, chờ lâu rồi...",web-playfpt
1,555457bf1a77ce5d65ea0617,130,event,552b2f0817dc135c89d7d6a2,Chờ đợi chương trình này đã lâu! Hẹn mọi người...,web-playfpt
2,555413001a77ce5d65ea0609,527201,vod,5453639f17dc1371f538a4cd,comment cai,android
3,555357f71a77ce5d65ea0606,424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt
4,555491181a77ce5d65ea0636,805,vod,5549cace17dc132a0cee1801,Phim rất hay ;),android


In [79]:
# pre-process data
test = 'Phim đck k không, ko dc đc được hay?'

# dùng regex để standardize
def standardize_data(row):
    # chuyển viết hoa thành viết thường
    row = row.lower()
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")

    # Standardize từ viết tắt như ko, đc, vn
    row = re.sub("ko? ","không ", row)
    row = re.sub("[dđ]c","được", row)
    row = re.sub("vn","việt nam", row)
    row = re.sub("roi","rồi", row)
    
    # Xóa các space ở đầu và cuối câu
    row = row.strip()
    return row
# standardize câu test
print(standardize_data(test))

df['content'] = df['content'].astype(str)
df['content_std'] = df['content'].apply(standardize_data)

df.head(5)

phim đượckhông không không  không được được được hay


,_id,user_id,comment_on,object_id,content,device,content_std
0,555466761a77ce5d65ea062a,12,event,552b2f0817dc135c89d7d6a2,"Chương trình \ Tuyệt cú mèo\ luôn, chờ lâu rồi...",web-playfpt,chương trình \ tuyệt cú mèo\ luôn chờ lâu rồi...
1,555457bf1a77ce5d65ea0617,130,event,552b2f0817dc135c89d7d6a2,Chờ đợi chương trình này đã lâu! Hẹn mọi người...,web-playfpt,chờ đợi chương trình này đã lâu hẹn mọi người...
2,555413001a77ce5d65ea0609,527201,vod,5453639f17dc1371f538a4cd,comment cai,android,comment cai
3,555357f71a77ce5d65ea0606,424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...
4,555491181a77ce5d65ea0636,805,vod,5549cace17dc132a0cee1801,Phim rất hay ;),android,phim rất hay )


In [29]:
# load pre trained model
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

# load Tokenizer từ model (dùng để tách từ trong câu)
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)


In [76]:
# test câu
sentence = 'cũng được'

# tách câu thành list từng từ
input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    # predict output bằng pretrained model
    out = pd.Series(out.logits.softmax(dim=-1).tolist()[0], index= ['NEG', 'POS', 'NEU'])
    print(out)


tensor([[ 0, 32, 11,  2]])
NEG    0.021827
POS    0.216084
NEU    0.762088
dtype: float64


In [18]:
# hàm để đánh giá từng comment
def sentiment(sentence):
    try:
        input_ids = torch.tensor([tokenizer.encode(sentence)])
    except:
        return 2

    if input_ids.shape[1] > tokenizer.model_max_length:
        return 2

    with torch.no_grad():
        out = model(input_ids)
        return np.argmax(out.logits.softmax(dim=-1).tolist()[0])

print(sentiment(sentence))

2


In [77]:
import time
start_time = time.time()
# test 1000 first comments
test = df[:999]
# apply tất cả comment
test['sentiment'] = test['content_std'].apply(sentiment)

print("--- %s seconds ---" % (time.time() - start_time))
test

--- 49.572404861450195 seconds ---


M:\TEMP\ipykernel_13216\3620050311.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sentiment'] = test['content_std'].apply(sentiment)


,_id,user_id,comment_on,object_id,content,device,content_std,sentiment
0,555466761a77ce5d65ea062a,12,event,552b2f0817dc135c89d7d6a2,"Chương trình \ Tuyệt cú mèo\ luôn, chờ lâu rồi...",web-playfpt,chương trình \ tuyệt cú mèo\ luôn chờ lâu rồi...,1
1,555457bf1a77ce5d65ea0617,130,event,552b2f0817dc135c89d7d6a2,Chờ đợi chương trình này đã lâu! Hẹn mọi người...,web-playfpt,chờ đợi chương trình này đã lâu hẹn mọi người...,1
2,555413001a77ce5d65ea0609,527201,vod,5453639f17dc1371f538a4cd,comment cai,android,comment cai,1
3,555357f71a77ce5d65ea0606,424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...,0
4,555491181a77ce5d65ea0636,805,vod,5549cace17dc132a0cee1801,Phim rất hay ;),android,phim rất hay ),1
...,...,...,...,...,...,...,...,...
994,556211d21a77ce110a495da3,532136,vod,5514cf3117dc135c57f3c612,dở quá chịu ko nỗi,android,dở quá chịu không nỗi,0
995,556211db1a77ce110a495da4,532136,vod,5514cf3117dc135c57f3c612,dở quá chịu ko nỗi,android,dở quá chịu không nỗi,0
996,55628bdc1a77ce110a495db6,372844,vod,554872c117dc132a0cee1794,coi phim buồn mấy cũng phải cười vỡ bụng,web-playfpt,coi phim buồn mấy cũng phải cười vỡ bụng,0
997,5562cd4a1a77ce110a495dcc,132963,vod,555db80d17dc1321b24f9d54,"hayvbnnnmmm,",ios,hayvbnnnmmm,2


In [80]:
# apply tất cả comment
df['sentiment'] = df['content_std'].apply(sentiment)

print("--- %s seconds ---" % (time.time() - start_time))
df

--- 47458.41684007645 seconds ---


,_id,user_id,comment_on,object_id,content,device,content_std,sentiment
0,555466761a77ce5d65ea062a,12,event,552b2f0817dc135c89d7d6a2,"Chương trình \ Tuyệt cú mèo\ luôn, chờ lâu rồi...",web-playfpt,chương trình \ tuyệt cú mèo\ luôn chờ lâu rồi...,1
1,555457bf1a77ce5d65ea0617,130,event,552b2f0817dc135c89d7d6a2,Chờ đợi chương trình này đã lâu! Hẹn mọi người...,web-playfpt,chờ đợi chương trình này đã lâu hẹn mọi người...,1
2,555413001a77ce5d65ea0609,527201,vod,5453639f17dc1371f538a4cd,comment cai,android,comment cai,1
3,555357f71a77ce5d65ea0606,424467,vod,551a43b417dc135c58f3c691,"lồng tiếng mà như tập đọc, nghe không nuốt nổi...",web-playfpt,lồng tiếng mà như tập đọc nghe không nuốt nổi...,0
4,555491181a77ce5d65ea0636,805,vod,5549cace17dc132a0cee1801,Phim rất hay ;),android,phim rất hay ),1
...,...,...,...,...,...,...,...,...
885164,627543d15e052026df2928c6,14734162,NaN,607d21222089bd064e20bcaa,"phim nhẹ nhàng dễ thương, ko có gì gây cấn ...",android,phim nhẹ nhàng dễ thương không có gì gây c...,1
885165,6136ee5e5e0520b679234d52,11069002,NaN,60111c9a2089bd03e099a609,"hay lắm bạn, xem đi ko uổng đâu",android,hay lắm bạn xem đi không uổng đâu,1
885166,627a3dc95e052026df2928c9,19094650,NaN,627a1a292089bd00e975ceff,Năm nay thấy đội nào đá cũng hay Myanmar hay M...,android,năm nay thấy đội nào đá cũng hay myanmar hay m...,2
885167,627e52f45e052026df2928ce,19094650,NaN,627dc28e2089bd02ce6079e1,U23 Myanmar đá k khác gì như Indo có điều ít c...,android,u23 myanmar đá không khác gì như indo có điều ...,0


In [ ]:
df.to_csv('sentiment.csv', index= False)